In [5]:
import numpy as np
from cities.utils.data_grabber import find_repo_root, MSADataGrabberCSV, DataGrabberCSV
import pandas as pd
import numpy as np
import requests
from us import states

root = find_repo_root()

data = DataGrabberCSV()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]

dataMA = MSADataGrabberCSV()
dataMA.get_features_wide(["gdp_ma"])
gdp_ma = dataMA.wide["gdp_ma"]


# note that there is also a library for accessing the Census data:
# https://github.com/datamade/census


# description of the missing values in the data (it is sometimes caused by a mistake that can be corrected):
#https://www2.census.gov/geo/pdfs/reference/Geography_Notes.pdf

In [13]:
variables = "NAME,DP05_0070E,DP05_0072E,DP05_0073E,DP05_0074E,DP05_0075E,DP05_0077E,DP05_0078E,DP05_0079E,DP05_0080E,DP05_0081E,DP05_0082E,DP05_0083E"

county_fips = "*" # all counties
tract = "*" # all tracts
api_key = '077d857d6c12d5b9b3aeafa07d2c1916ba12a86c' # private api key required to access the data https://api.census.gov/data/key_signup.html
year = 2019

dfs = []

for x in range(0, len(states.STATES)): # in this call it's not possible to use the '*' wildcard to access all states, so we need to iterate over all states
    fips = states.STATES[x].fips

    url = f'https://api.census.gov/data/{year}/acs/acs5/profile?get={variables}&for=tract:{tract}&in=state:{fips}&in=county:{county_fips}&key={api_key}'

    response = requests.get(url)

    assert response.status_code == 200, 'The data retrieval went wrong'  # 200 means success

    print(f'{fips} fips done')

    data = response.json()

    df = pd.DataFrame(data[1:], columns=data[0])

    dfs.append(df)
    
    combined_df_2019 = pd.concat(dfs, ignore_index=True)




01 fips done
02 fips done
04 fips done
05 fips done
06 fips done
08 fips done
09 fips done
10 fips done
12 fips done
13 fips done
15 fips done
16 fips done
17 fips done
18 fips done
19 fips done
20 fips done
21 fips done
22 fips done
23 fips done
24 fips done
25 fips done
26 fips done
27 fips done
28 fips done
29 fips done
30 fips done
31 fips done
32 fips done
33 fips done
34 fips done
35 fips done
36 fips done
37 fips done
38 fips done
39 fips done
40 fips done
41 fips done
42 fips done
44 fips done
45 fips done
46 fips done
47 fips done
48 fips done
49 fips done
50 fips done
51 fips done
53 fips done
54 fips done
55 fips done
56 fips done


In [14]:
# Different variable codes! As the definitions changed

variables = "NAME,DP05_0072E,DP05_0074E,DP05_0075E,DP05_0076E,DP05_0077E,DP05_0079E,DP05_0080E,DP05_0081E,DP05_0082E,DP05_0083E,DP05_0084E,DP05_0085E"
county_fips = "*" # all counties
tract = "*" # all tracts
api_key = '077d857d6c12d5b9b3aeafa07d2c1916ba12a86c' # private api key required to access the data https://api.census.gov/data/key_signup.html
year = 2022

dfs = []

for x in range(0, len(states.STATES)): # in this call it's not possible to use the '*' wildcard to access all states, so we need to iterate over all states
    fips = states.STATES[x].fips

    url = f'https://api.census.gov/data/{year}/acs/acs5/profile?get={variables}&for=tract:{tract}&in=state:{fips}&in=county:{county_fips}&key={api_key}'

    response = requests.get(url)

    assert response.status_code == 200, 'The data retrieval went wrong'  # 200 means success

    print(f'{fips} fips done')

    data = response.json()

    df = pd.DataFrame(data[1:], columns=data[0])

    dfs.append(df)
    
    combined_df_2022 = pd.concat(dfs, ignore_index=True)




01 fips done
02 fips done
04 fips done
05 fips done
06 fips done
08 fips done
09 fips done
10 fips done
12 fips done
13 fips done
15 fips done
16 fips done
17 fips done
18 fips done
19 fips done
20 fips done
21 fips done
22 fips done
23 fips done
24 fips done
25 fips done
26 fips done
27 fips done
28 fips done
29 fips done
30 fips done
31 fips done
32 fips done
33 fips done
34 fips done
35 fips done
36 fips done
37 fips done
38 fips done
39 fips done
40 fips done
41 fips done
42 fips done
44 fips done
45 fips done
46 fips done
47 fips done
48 fips done
49 fips done
50 fips done
51 fips done
53 fips done
54 fips done
55 fips done
56 fips done


In [15]:
combined_df_2019.head(2)

,NAME,DP05_0070E,DP05_0072E,DP05_0073E,DP05_0074E,DP05_0075E,DP05_0077E,DP05_0078E,DP05_0079E,DP05_0080E,DP05_0081E,DP05_0082E,DP05_0083E,state,county,tract
0,"Census Tract 38.02, Jefferson County, Alabama",5291,0,0,0,0,85,5161,0,0,0,0,45,01,073,003802
1,"Census Tract 11, Jefferson County, Alabama",4781,17,0,0,0,77,4629,58,0,0,0,0,01,073,001100


In [16]:
combined_df_2022.head(2)

,NAME,DP05_0072E,DP05_0074E,DP05_0075E,DP05_0076E,DP05_0077E,DP05_0079E,DP05_0080E,DP05_0081E,DP05_0082E,DP05_0083E,DP05_0084E,DP05_0085E,state,county,tract
0,Census Tract 201; Autauga County; Alabama,1865,46,0,0,35,1428,208,0,45,0,0,103,01,001,020100
1,Census Tract 202; Autauga County; Alabama,1861,2,0,0,0,674,1042,0,0,0,8,135,01,001,020200


In [102]:
ethnic_pre2020 = combined_df_2019.copy()
ethnic_post2020 = combined_df_2022.copy()

In [103]:
column_mapping_2019 = {
    'DP05_0070E': 'total_pop',
    'DP05_0072E': 'mexican',
    'DP05_0073E': 'puerto_rican',
    'DP05_0074E': 'cuban',
    'DP05_0075E': 'other_hispanic_latino',
    'DP05_0077E': 'white',
    'DP05_0078E': 'black_african_american',
    'DP05_0079E': 'american_indian_alaska_native',
    'DP05_0080E': 'asian',
    'DP05_0081E': 'native_hawaiian_other_pacific_islander',
    'DP05_0082E': 'other_race',
    'DP05_0083E': 'two_or_more_sum'
}


column_mapping_2022 = {
    'DP05_0072E': 'total_pop', # those variable names work for 2022, be aware that in other years their meaning may differ
    'DP05_0074E': 'mexican',
    'DP05_0075E': 'puerto_rican',
    'DP05_0076E': 'cuban',
    'DP05_0077E': 'other_hispanic_latino',
    'DP05_0079E': 'white',
    'DP05_0080E': 'black_african_american',
    'DP05_0081E': 'american_indian_alaska_native',
    'DP05_0082E': 'asian',
    'DP05_0083E': 'native_hawaiian_other_pacific_islander',
    'DP05_0084E': 'other_race',
    'DP05_0085E': 'two_or_more_sum'
}


ethnic_pre2020.rename(columns=column_mapping_2019, inplace=True)
ethnic_pre2020.head(2)

,NAME,total_pop,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race,two_or_more_sum,state,county,tract
0,"Census Tract 38.02, Jefferson County, Alabama",5291,0,0,0,0,85,5161,0,0,0,0,45,01,073,003802
1,"Census Tract 11, Jefferson County, Alabama",4781,17,0,0,0,77,4629,58,0,0,0,0,01,073,001100


In [104]:
ethnic_post2020.rename(columns=column_mapping_2022, inplace=True)
ethnic_post2020.head(2)

,NAME,total_pop,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race,two_or_more_sum,state,county,tract
0,Census Tract 201; Autauga County; Alabama,1865,46,0,0,35,1428,208,0,45,0,0,103,01,001,020100
1,Census Tract 202; Autauga County; Alabama,1861,2,0,0,0,674,1042,0,0,0,8,135,01,001,020200


In [105]:
ethnic_list = [ethnic_pre2020, ethnic_post2020]   

state_abbreviations = {
        'Alabama': 'AL',
        'Alaska': 'AK',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY'
    }

def parse_geo_name(name):
    if ';' in name:
        parts = name.split(';')
    else:
        parts = name.split(',')

    if len(parts) >= 3:
        county = parts[1].strip().replace(' County', '')
        state_full = parts[2].strip()
        state_abbr = state_abbreviations.get(state_full, state_full)
        return f"{county}, {state_abbr} (CT)"
    return "Unknown"

for i in range(len(ethnic_list)):
    ethnic = ethnic_list[i]
    
    # Create GeoFIPS column
    ethnic['GeoFIPS'] = ethnic.apply(lambda row: f"{row['state']}{row['county']}{row['tract']}", axis=1).astype(np.int64)
    
    # Drop unnecessary columns
    ethnic.drop(['state', 'county', 'tract'], axis=1, inplace=True)
    
    # Apply parse_geo_name to create GeoName column
    ethnic['GeoName'] = ethnic['NAME'].apply(parse_geo_name).astype(str)
    
    # Ensure no unknown GeoNames
    assert ethnic[ethnic['GeoName'] == 'Unknown'].shape[0] == 0, 'There are Unknown GeoNames'
    
    # Drop NAME column
    ethnic.drop(['NAME'], axis=1, inplace=True)
    
    # Print unique GeoNames and head of dataframe
    print(ethnic['GeoName'].nunique())
    
    # Store the processed dataframe back in the list
    ethnic_list[i] = ethnic

# Assign processed dataframes back to original variable names
ethnic_pre2020, ethnic_post2020 = ethnic_list

3141
3143


In [106]:
ethnic_pre2020.head(2)

,total_pop,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race,two_or_more_sum,GeoFIPS,GeoName
0,5291,0,0,0,0,85,5161,0,0,0,0,45,1073003802,"Jefferson, AL (CT)"
1,4781,17,0,0,0,77,4629,58,0,0,0,0,1073001100,"Jefferson, AL (CT)"


In [107]:
ethnic_post2020.head(2)

,total_pop,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race,two_or_more_sum,GeoFIPS,GeoName
0,1865,46,0,0,35,1428,208,0,45,0,0,103,1001020100,"Autauga, AL (CT)"
1,1861,2,0,0,0,674,1042,0,0,0,8,135,1001020200,"Autauga, AL (CT)"


In [108]:
ethnic_list = [ethnic_pre2020, ethnic_post2020]  


for i in range(len(ethnic_list)):
    ethnic = ethnic_list[i]
    
    rows1 = ethnic.shape[0]
    ethnic.dropna(how='any', inplace=True)  # Drop NaN values inplace
    rows2 = ethnic.shape[0]
    print(f"This many rows were removed because of NaNs: {rows1 - rows2}")
    
    ethnic.sort_values(by=['GeoFIPS', 'GeoName'], inplace=True)
    
    cols_to_save = ethnic.shape[1] - 2
    ethnic = ethnic[['GeoFIPS', 'GeoName'] + list(ethnic.columns[0:cols_to_save])]
    
    # Update the DataFrame in the list
    ethnic_list[i] = ethnic.reset_index(drop=True)

ethnic_pre2020, ethnic_post2020 = ethnic_list


This many rows were removed because of NaNs: 0
This many rows were removed because of NaNs: 0


In [96]:
# read the areas post2020


# and

# df['CBSA'] = df['CBSA'].astype(np.int64)
# df2 = df[df['CBSA'].isin(metro_areas['GeoFIPS'])]

# df2 = pd.merge(df2, metro_areas[['GeoFIPS', 'GeoName']], left_on='CBSA', right_on='GeoFIPS', how='inner')
# df2 = df2.drop_duplicates(subset=['CBSA'])

# df2.drop(columns='CBSA', inplace=True)
# df2.head()



In [109]:
# wrangling other races, verifying the sum to the total population
# normalizing the values, to create a composition of the population

for i in range(len(ethnic_list)):
    ethnic = ethnic_list[i]
    
    ethnic.iloc[:, 2:] = ethnic.iloc[:, 2:].apply(pd.to_numeric, errors="coerce")
    ethnic[ethnic.columns[2:]] = ethnic[ethnic.columns[2:]].astype(float)

    ethnic["other_race_races"] = ethnic["other_race"] + ethnic["two_or_more_sum"]

    ethnic = ethnic.drop(["other_race", "two_or_more_sum"], axis=1)
    
    ethnic["totalALT"] = ethnic.iloc[:, 3:].sum(axis=1)
    assert (ethnic["totalALT"] == ethnic["total_pop"]).all()

    ethnic = ethnic.drop("totalALT", axis=1)

    row_sums = ethnic.iloc[:, 2:].sum(axis=1)
    ethnic.iloc[:, 3:] = ethnic.iloc[:, 3:].div(row_sums, axis=0)
    
    ethnic = ethnic.drop(["total_pop"], axis=1)

    
    ethnic_list[i] = ethnic

ethnic_pre2020, ethnic_post2020 = ethnic_list


/tmp/ipykernel_9948/3620330864.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ethnic.iloc[:, 2:] = ethnic.iloc[:, 2:].apply(pd.to_numeric, errors="coerce")
/tmp/ipykernel_9948/3620330864.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  ethnic.iloc[:, 2:] = ethnic.iloc[:, 2:].apply(pd.to_numeric, errors="coerce")


In [110]:
for i in ethnic_list:
    print(i.dtypes)

GeoFIPS                                     int64
GeoName                                    object
mexican                                   float64
puerto_rican                              float64
cuban                                     float64
other_hispanic_latino                     float64
white                                     float64
black_african_american                    float64
american_indian_alaska_native             float64
asian                                     float64
native_hawaiian_other_pacific_islander    float64
other_race_races                          float64
dtype: object
GeoFIPS                                     int64
GeoName                                    object
mexican                                   float64
puerto_rican                              float64
cuban                                     float64
other_hispanic_latino                     float64
white                                     float64
black_african_american              

In [112]:
ethnic_pre2020

,GeoFIPS,GeoName,mexican,puerto_rican,cuban,other_hispanic_latino,white,black_african_american,american_indian_alaska_native,asian,native_hawaiian_other_pacific_islander,other_race_races
0,1001020100,"Autauga, AL (CT)",0.004767,0.000000,0.0,0.001756,0.416207,0.038133,0.000000,0.000502,0.000000,0.038635
1,1001020200,"Autauga, AL (CT)",0.002297,0.005360,0.0,0.000000,0.191424,0.285094,0.000000,0.000000,0.000000,0.015824
2,1001020300,"Autauga, AL (CT)",0.033219,0.000000,0.0,0.000000,0.325777,0.122184,0.000000,0.004562,0.000713,0.013544
3,1001020400,"Autauga, AL (CT)",0.000000,0.000903,0.0,0.004513,0.448427,0.027849,0.004126,0.001418,0.000000,0.012764
4,1001020500,"Autauga, AL (CT)",0.015147,0.004341,0.0,0.002643,0.354521,0.091025,0.000000,0.018167,0.000000,0.014156
...,...,...,...,...,...,...,...,...,...,...,...,...
72872,56043000200,"Washakie, WY (CT)",0.030973,0.000000,0.0,0.001475,0.447067,0.000492,0.000656,0.000000,0.000000,0.019338
72873,56043000301,"Washakie, WY (CT)",0.098509,0.000000,0.0,0.012691,0.358985,0.000000,0.000000,0.000000,0.000000,0.029815
72874,56043000302,"Washakie, WY (CT)",0.065357,0.001604,0.0,0.011628,0.413793,0.000000,0.007618,0.000000,0.000000,0.000000
72875,56045951100,"Weston, WY (CT)",0.007893,0.000455,0.0,0.000304,0.469642,0.002429,0.000304,0.005313,0.000000,0.013661


In [ ]:
# save them

ethnic_pre2020.to_csv(f"{root}/data/raw/ethnic_composition_pre2020_filtered_wide.csv", index=False)
ethnic_post2020.to_csv(f"{root}/data/raw/ethnic_composition_post2020_filtered_wide.csv", index=False)